In [1]:
import os
import argparse
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
train_record_file = "/home/maoransu/SSD_Mobile/models/research/object_detection/datasets/CURB_TF/curb_train.record"
val_record_file = "/home/maoransu/SSD_Mobile/models/research/object_detection/datasets/CURB_TF/curb_val.record"
save_path = '/home/maoransu/yolo/darknet/build/darknet/x64/data/curb'

In [3]:
def parse_func(serialized_example):
    features = tf.io.parse_single_example(serialized_example,
                                       features={
                        'image/height': tf.io.FixedLenFeature([], tf.int64),
                        'image/width': tf.io.FixedLenFeature([], tf.int64),
                        'image/encoded': tf.io.FixedLenFeature([], tf.string),
                        'image/format': tf.io.FixedLenFeature([], tf.string),
                        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
                        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
                        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
                        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
                        'image/object/class/text': tf.io.VarLenFeature(tf.string),
                        'image/object/class/label': tf.io.VarLenFeature(tf.int64)
    })
    image = features['image/encoded']
    image_format = features['image/format']
    height = tf.cast(features['image/height'], tf.int32)
    width = tf.cast(features['image/height'], tf.int32)
    xmin = tf.sparse.to_dense(features['image/object/bbox/xmin'])
    xmax = tf.sparse.to_dense(features['image/object/bbox/xmax'])
    ymin = tf.sparse.to_dense(features['image/object/bbox/ymin'])
    ymax = tf.sparse.to_dense(features['image/object/bbox/ymax'])
    label = tf.sparse.to_dense(features['image/object/class/label'])
    return image, image_format, height, width, xmin, xmax, ymin, ymax, label

In [4]:
def convert(record_path, subset, save_path):
    '''
    record_path: path to tfrecord file to be converted
    subset: train or val
    save_path: folder to save the images and txt
    '''
    dataset = tf.data.TFRecordDataset([record_path])
    dataset = dataset.map(parse_func)
    iterator = dataset.__iter__()
    
    subset_list = []
    
    for idx, sample in enumerate(iterator):
        (image, image_format, height, width,
                 xmin, xmax, ymin, ymax, label) = sample
        with open('{0}/{1}_{2}.jpg'.format(save_path, subset, idx), 'wb') as f:
            f.write(image.numpy())

        with open('{0}/{1}_{2}.txt'.format(save_path, subset, idx), 'w') as f:
            for x1, x2, y1, y2, l in zip(xmin, xmax, ymin, ymax, label):
                print('{} {} {} {} {}'.format(l-1, (x1+x2)/2,
                                              (y1+y2)/2, (x2-x1),
                                              (y2-y1)), file=f)
        subset_list.append('data/curb/{}_{}.jpg'.format(subset, idx))

    with open('{0}/../{1}.txt'.format(save_path, subset), 'w') as f:
        for name in subset_list:
            f.write(name+'\n')
            

In [5]:
convert(train_record_file, 'train', save_path)
convert(val_record_file, 'val', save_path)

In [35]:
dataset = tf.data.TFRecordDataset([val_record_file])
dataset = dataset.map(parse_func)
iterator = dataset.__iter__()
next_element = next(iterator)

In [6]:
(image, image_format, height, width,
             xmin, xmax, ymin, ymax, label) = next_element

In [36]:
sum(1 for _ in iterator)

251

In [20]:
with open('train_0.jpg', 'wb') as f:
    f.write(image.numpy())